<a href="https://colab.research.google.com/github/harshi681/LAB2/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.6 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
import faiss
import numpy as np
from transformers import pipeline

In [4]:
# Initialize embedding model and vector index
model = SentenceTransformer('all-MiniLM-L6-v2')  # Pre-trained embedding model
dimension = 384  # Embedding dimension for MiniLM
index = faiss.IndexFlatL2(dimension)  # FAISS flat index
metadata_store = []  # Store chunks and metadata

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def crawl_and_scrape(url):
    """Crawl and scrape content from a webpage."""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = ' '.join([p.text for p in soup.find_all('p')])  # Extract paragraph texts
    return text

In [6]:
def chunk_text(text, chunk_size=500):
    """Divide text into manageable chunks."""
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

def process_and_store(url):
    """Crawl, scrape, chunk, and store data in the vector database."""
    global metadata_store
    text = crawl_and_scrape(url)
    chunks = chunk_text(text)
    embeddings = model.encode(chunks)
    faiss.normalize_L2(embeddings)
    index.add(np.array(embeddings))
    metadata_store.extend([{'text': chunk} for chunk in chunks])
    print(f"Processed {len(chunks)} chunks from {url}")

def query_database(query, top_k=3):
    """Search the vector database for relevant chunks."""
    query_embedding = model.encode([query])
    faiss.normalize_L2(query_embedding)
    distances, indices = index.search(query_embedding, top_k)
    results = [metadata_store[idx]['text'] for idx in indices[0] if idx < len(metadata_store)]
    return results

In [7]:
# Initialize the language model pipeline
llm = pipeline("text2text-generation", model="google/flan-t5-base")

def generate_response(query, retrieved_texts):
    """Generate a response using the language model."""
    context = "\n".join(retrieved_texts)
    prompt = f"Based on the following information, answer the query:\n\n{context}\n\nQuery: {query}\n\nAnswer:"
    response = llm(prompt, max_length=300, num_return_sequences=1)
    return response[0]['generated_text']


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [11]:
# Main workflow
if __name__ == "__main__":
    url = input("Enter the URL to scrape: ")
    try:
        process_and_store(url)

        query = input("Enter your query: ")
        relevant_chunks = query_database(query)
        if relevant_chunks:
            response = generate_response(query, relevant_chunks)
            print("Response:", response)
        else:
            print("No relevant information found.")
    except Exception as e:
        print(f"An error occurred: {e}")


Enter the URL to scrape: https://en.wikipedia.org/wiki/Natural_language_processing
Processed 16 chunks from https://en.wikipedia.org/wiki/Natural_language_processing
Enter your query: What is natural language processing?
Response: Natural language processing (NLP) is a subfield of computer science and especially artificial intelligence.


In [ ]:
_